# Project 3: Data Cleaning - Tidy up messy Datasets (Movies Dataset)

# Project Brief for Self-Coders

Here you´ll have the opportunity to code major parts of Project 3 on your own. If you need any help or inspiration, have a look at the Videos or the Jupyter Notebook with the full code. <br> <br>
Keep in mind that it´s all about __getting the right results/conclusions__. It´s not about finding the identical code. Things can be coded in many different ways. Even if you come to the same conclusions, it´s very unlikely that we have the very same code. 

In [1]:
import pandas as pd
import json
import ast
import numpy as np
pd.options.display.max_columns =25



## First Steps 

1. __Load__ and __inspect__ the messy dataset __movies_metadata.csv__. Identify columns with nested / stringified json data.

In [2]:
df = pd.read_csv('movies_metadata.csv', low_memory=False)

In [ ]:

# columns with JSON mixed: 
# 'belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'

## Dropping irrelevant Columns

2. __Drop__ the irrelevant columns 'adult', 'imdb_id', 'original_title', 'video' and 'homepage'.

In [3]:
df = df.drop(columns= ['adult', 'imdb_id', 'original_title', 'homepage'], axis=1)

In [ ]:
df.info()

## How to handle stringified JSON columns

3. __Evaluate__ Python Expressions in the stringified columns ["belongs_to_collection", "genres", "production_countries", "production_companies", "spoken_languages"] and __remove quotes__ ("") where possible.

In [ ]:
json_example = df.belongs_to_collection[2]

json.loads(
#     .replace "" to '', json.loads can't work with ""
    json_example.replace("'", '"')
    )

# using this method, we can start apply to work with JSON in dataframe

In [ ]:
df.genres.apply( lambda x: json.loads(x.replace("'", '"')))#[0]

# Note that::
# belongs_to_collection has mixed types.
#  .replace() encounter errors when it run thru those mixed types

In [ ]:
df.genres.apply(lambda x: ast.literal_eval(x))
# ast.literal_eval() can read json with "", skiping a step from complication of "" and '' that json.loads has
# but ast.literal_eval still can't read mixed types as in belongs_to_collection

In [4]:
df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: ast.literal_eval(x) 
                               if isinstance(x, str)
                               else np.nan)

df.genres = df.genres.apply(lambda x: ast.literal_eval(x) 
                               if isinstance(x, str)
                               else np.nan)

df.production_companies = df.production_companies.apply(lambda x: ast.literal_eval(x) 
                               if isinstance(x, str)
                               else np.nan)

df.production_countries = df.production_countries.apply(lambda x: ast.literal_eval(x) 
                               if isinstance(x, str)
                               else np.nan)

df.spoken_languages = df.spoken_languages.apply(lambda x: ast.literal_eval(x) 
                               if isinstance(x, str)
                               else np.nan)

# translation: if the cells is instance 'str', then apply ast.literal_eval. Else, change it to np.nand

In [15]:
df.head(30)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation|Comedy|Family,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,NaN,65000000,Adventure|Fantasy|Family,8844,en,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance|Comedy,15602,en,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,NaN,16000000,Comedy|Drama|Romance,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,NaN,60000000,Action|Crime|Drama|Thriller,949,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,NaN,58000000,Comedy|Romance,11860,en,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,NaN,0,Action|Adventure|Drama|Family,45325,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,NaN,35000000,Action|Adventure|Thriller,9091,en,International action superstar Jean Claude Van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,Adventure|Action|Thriller,710,en,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT

## How to flatten nested Columns

4. __Extract__ only the __collection name__ from the column "belongs_to_collection" and __overwrite__ "belongs_to_collection". <br> For example: The value in the first row (Toy Story) should be 'Toy Story Collection'.

In [18]:
# df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: x['name']
#                               if isinstance(x, dict)
#                               else np.nan)

df.belongs_to_collection.apply(lambda x: type(x))

0         <class 'dict'>
1        <class 'float'>
2         <class 'dict'>
3        <class 'float'>
4         <class 'dict'>
              ...       
45461    <class 'float'>
45462    <class 'float'>
45463    <class 'float'>
45464    <class 'float'>
45465    <class 'float'>
Name: belongs_to_collection, Length: 45466, dtype: object

5. __Extract__ all __genre names__ from the column "genres" and __overwrite__ "genres". If a movie has more than one genre, __seperate genres by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Animation|Comedy|Family'.

In [9]:
df.genres = df.genres.apply(lambda x: "|".join(i['name'] for i in x))

6. __Extract__ all __spoken language names__ from the column "spoken_languages" and __overwrite__ "spoken_languages". If a movie has more than one spoken language, __seperate spoken languages by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'English'.

In [17]:
# df.spoken_languages.apply(lambda x: "|".join(i['name'] for i in x))

df.spoken_languages.apply(lambda x: type(x))

0        <class 'list'>
1        <class 'list'>
2        <class 'list'>
3        <class 'list'>
4        <class 'list'>
              ...      
45461    <class 'list'>
45462    <class 'list'>
45463    <class 'list'>
45464    <class 'list'>
45465    <class 'list'>
Name: spoken_languages, Length: 45466, dtype: object

7. __Extract__ all __production countries names__ from the column "production_countries" and __overwrite__ "production_countries". If a movie has more than one production country, __seperate production countries by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'United States of America'.

8. __Extract__ all __production companies names__ from the column "production_companies" and __overwrite__ "production_companies". If a movie has more than one production company, __seperate production companies by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Pixar Animation Studios'

9. __Inspect__ all columns above with value_counts(). Do you see anything strange? __Take reasonable measures__!

In [12]:
df.genres.value_counts()

Drama                              5000
Comedy                             3621
Documentary                        2723
                                   2442
Drama|Romance                      1301
                                   ... 
Action|Drama|Comedy|Documentary       1
War|Drama|History|Thriller            1
Horror|Drama|History|Thriller         1
Comedy|Crime|Action|Drama             1
Family|Animation|Romance|Comedy       1
Name: genres, Length: 4069, dtype: int64

## Cleaning Numerical Columns

10. __Convert__ the datatype in the columns __"budget"__, __"id"__ and __"popularity"__ __to numeric__. Set invalid values as NaN.

11. __Analyze__ the columns __"budget"__ and __"revenue"__ and __"runtime"__. Analyze movies with a budget/revenue/runtime of 0. Do you think the value 0 is the most appropriate value? __Take reasonable measures__! 

12. The columns "budget" and "revenue" shall show values in Million USD. __Convert and Overwrite__!

13. __Analyze__ movies with a __vote_count of 0__. What´s the __vote_average__ for those movies? Do you think this value is the most appropriate value? __Take reasonable measures__!

## Cleaning DateTime Columns

14. __Convert__ the datatype in the column __"release_date"__ __to datetime__. Set invalid values as NaN.

## Cleaning Text / String Columns

15. __Analyze__ the text columns "overview" and "tagline". Try to identify __missing data that is not represented by NaN__ (e.g. "No Data"). __Replace as NaN__ (np.nan)!

## Removing Duplicates

16. __Identify__ and __remove__ duplicates!

## Handling Missing Values & Removing Observations

17. __Drop__ all rows/movies with unknown __id__ or __title__.

18. __Keep__ only those rows/movies in the df with __10 or more non-NaN__ values.

## Final (Cleaning) Steps

19. __Keep__ only those rows/movies in the df with __status "Released"__. Then __drop__ the column "status".

20. The Order of the columns should be as follows: 

In [ ]:
["id", "title", "tagline", "release_date", "genres", "belongs_to_collection", 
"original_language", "budget_musd", "revenue_musd", "production_companies",
"production_countries", "vote_count", "vote_average", "popularity", "runtime",
"overview", "spoken_languages", "poster_path"]

21. __Reset__ the Index and create a __RangeIndex__.

22. __Save__ the cleaned dataset in a __csv-file__.

# +++++++++ See some Hints below +++++++++++++

# ++++++++++++++++ Hints++++++++++++++++++++

__Hints for 3.__ <br>
apply ast.literal_eval() on all stringified elements (you have to import ast):

In [ ]:
# example:
df.stringified_column = df.stringified_column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

__Hints for 4., 5., 6., 7., 8.__<br> 
apply an appropriate lambda function on all column elements

__Hints for 9.__<br>
Replace all __""__ (empty strings) in the above columns by NaN (__np.nan__)

__Hints for 10.__<br>
Use pd.to_numeric() and "coerce" errors

__Hints for 11.__<br>
Replace the value 0 by NaN (__np.nan__)

__Hints for 13.__<br>
Replace the value 0 by NaN (__np.nan__)

__Hints for 14.__<br>
Use pd.to_datetime() and "coerce" errors

__Hints for 16.__<br>
There cannot be two or more movies with the same movie id.